<a href="https://colab.research.google.com/github/ahmedhamodien/Machine-Learning-/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Github clone

In [3]:
!git clone https://github.com/your_username/Machine_Learning.git
%cd face-mask-detection


Cloning into 'Machine_Learning'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'face-mask-detection'
/content


# Libraries

In [9]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image

from tensorflow.keras.applications.densenet import preprocess_input

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from transformers import ViTFeatureExtractor, ViTForImageClassification
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from tqdm.keras import TqdmCallback

# Data preprocessing